# Writting Correction
This notebook is a simple example that shows how to create a Guidance Program that corrects a given text on grammar, spelling and writing using a Chat Model.

## Installing dependencies
Lets install the necessary packages for this tutorial notebook.

In [ ]:
%pip install guidance ipywidgets openai python-dotenv IPython

First let us import the necessary OpenAI Key for this tutorial. Make sure to create a `.env` file in the root of this directory like so:
```
OPENAI_API_KEY=sk-...
```

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file for our API Keys

import guidance

## Define a simple Guidance Program
Now let us create a simple Guidance Program that we can input a text to correct based on our preferences.

In [ ]:
llm = guidance.llms.OpenAI("gpt-4")

correction_program = guidance('''
{{#system~}}
You are a friendly expert in writing. You exceed in Grammar, Spelling, Vocabulary and Style. Assist the user with correcting their writing.
Only correct text that is delimited by <text></text>.
{{~/system}}

{{#user~}}
I need assistance with my writing. Help correct any Grammar and Spelling errors and suggest ways to improve my writing such as suggesting a diverse vocabulary.
Here is the text I wrote:
<text>
{{text_input}}
</text>
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''', llm=llm)

As you might notice, the syntax for defining a Program in Guidance is vastly different to Frameworks like [Langchain](https://python.langchain.com/docs/get_started/introduction.html). The syntax appears more like a templating language, and you are right! The syntax was heavily inspired by [Handlebars](https://handlebarsjs.com/).

As of time of this writing, there was no direct documentation for Guidance. But in general most handlebar syntax works here too or behaves similar.

For Chat Programs that support System Messages, you may use `{{#system}}` and `{{/system}}` to define the system message.
User Inputs may be defined similarly with the `user` keyword: `{{#user}}` and `{{/user}}`. Assistants are similar with the `assistant` keyword `{{#assistant}}` and `{{/assistant}}`. The special character `~` is used to strip whitespaces. Depending on the position: after the line or before it.

Notice the use of variables with `{{text_input}}`. Variables may be added to the Program before runtime or at runtime through the generation of an assistant. In our case, we just want to pass the users text to correct.

The `{{gen}}` keyword tells the program to pause and generate the snippet. If no blocks are hidden through `{{#block hidden=True}}`, the previous context will be passed. In our case, we don't need that. We define the variable name to be `answer`, set a temperature of `0` because we want to prevent hallucinations and set our `max_tokens` to 500. If you were to use this in a real application, you might want to ensure the limit is not exceeded by the user and the resulting output. For example by splitting the text into chunks, limiting the input or controlling the max_tokens based on the token length of the users text.

## Run the program
First let us define a sample text to use. In this example, we have generated a text using GPT-4 with spelling and grammatic errors, as well as a "monotone" vocabulary. Feel free to test on your own text!

In [ ]:
text = '''
Hey guyz! So, I wanna talk to u bout dis new fone what just came out. Like, its litteraly the bomb! Its got these awesome futures that gonna blow youre mind! The screeen is huge and colors is soooo vibrant. Its like looking at a rainbow threw a telescope or sumthing. Battery life? Dont even get me startid. It lasts like for ages. U can prolly use it for a week wifout chargin.
'''

Now let's run our simple Program:

In [ ]:
correction_program(text_input=text)

As we can see, the Program corrects spelling, grammar and depending on the input, it also might do some fine rewriting to make the text more appealing. However, we may want more fine-grained control when we do inference based on our template. Maybe, we just want spelling & grammar corrections but not a rewrite of our wording.
Also, in unfortunate situations it may add the delimiter or undesired output. For that, let us adjust our Program a little.

In [ ]:
llm = guidance.llms.OpenAI("gpt-4")

correction_program = guidance('''
{{#system~}}
You are a friendly expert in writing. You exceed in Grammar, Spelling, Vocabulary and Style. Assist the user with correcting their writing.
Only correct text that is delimited by <text></text>. When responding corrected text, do not add <text></text>.
Only correct text based on: 
{{~#each correction_criteria}}
- {{this}}
{{~/each}}
{{~/system}}

{{#user~}}
I need assistance with my writing. Help correct any Grammar and Spelling errors and suggest ways to improve my writing such as suggesting a diverse vocabulary.
Here is the text I wrote:
<text>
{{text_input}}
</text>
{{~/user}}

{{#assistant~}}
Sure, I am happy to help you. Here is the corrected text I wrote:
{{~/assistant}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''', llm=llm)

Now we can pass a list with the things we want to correct. Notice how we used `{{#each}}` here to build a list with bulletpoints with `{{this}}` we can dump the content of the variable in the List. If we were to have a dictionary as the current item we could access the sub-variables by dumping `{{this.variable}}`.

In [ ]:
correction_program(text_input=text, correction_criteria=['grammar', 'spelling'])

## Building a little Correcting Application
Now that we have verified our Guidance Program to work and adjusted our Program to give us more control over the output, we can build a little application in the notebook. In production, this would be a User Interface or an API where the user can insert a text to be corrected based on provided Criteria.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# The Input Widget for the users Text
text_input = widgets.Textarea(
    value='',
    placeholder='Enter the Text you wish to correct...',
    disabled=False,
    rows=10
)

# Options for the checkboxes
options=['Grammar', 'Spelling', 'Style', 'Vocabulary']
checkboxes = [widgets.Checkbox(value=False, description=option) for option in options]
checkboxes_container = widgets.VBox(checkboxes)
checkboxes_label = widgets.Label(value='Select what you want to correct')

# The Submit Button
button = widgets.Button(description='Submit', button_style='primary')

# Layout Container for button and Input
button_input_container = widgets.VBox([text_input, button])

# Create an output widget to display the conversation history
output = widgets.Output()

def format_input(user, user_input):
    return f"## {user}: \n --- \n {user_input}"

# Function to handle the text input
def handle_submit(sender):
    with output:
        # Check which checkboxes are selected
        selected_options = [cb.description for cb in checkboxes if cb.value]

        user_input = text_input.value
        # Clear the input box for the next message
        text_input.value = ''
        # Display the user's input
        display(Markdown(format_input('Text', user_input)))
        result = correction_program(text_input=user_input, correction_criteria=selected_options)
        display(Markdown(format_input('Suggestion', result['answer'])))
with output:
    display(Markdown('# Writing Correction'))

# Link the function to the button input's click event
button.on_click(handle_submit)

# Display the widgets
display(output, checkboxes_label, checkboxes_container, button_input_container)

Now we have a neat little interface where we can paste our text and select what we want to correct! Here are some ideas how to further improve this little application:

- Expand the Program to explain some corrections and thought processes (e.g. Chain of Thought)
- Add more Options and add Explanations in the System Message (e.g. Temporal Coherence)

Guidance offers a lot more features and customization abilities as this notebook was just made as a simple introduction. Feel free to check out the [repo](https://github.com/microsoft/guidance)!